In [9]:
# Import the relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


In [10]:
sustainability_data = pd.read_csv('data/datos_sostenibilidad.csv')
sustainability_data.head()

,hotel_nombre,consumo_energia_kwh,residuos_generados_kg,porcentaje_reciclaje,uso_agua_m3,fecha
0,Alletra Diamond Grand Hotel,3077,1821,73.5,4977,2019-01-01
1,ProLiant Towers,2253,1319,71.5,3145,2019-01-01
2,Aruba Luxury Lodge,3824,1672,89.7,4179,2019-01-01
3,InfoSight Boutique Hotel,1701,1070,82.7,4911,2019-01-01
4,Primera Grand,2970,1547,94.6,3907,2019-01-01


In [11]:
turistic_opinion = pd.read_csv('data/opiniones_turisticas.csv')
turistic_opinion = turistic_opinion.drop(columns=['idioma'])
hotel_opinions = turistic_opinion[turistic_opinion['tipo_servicio'] == 'Hotel']
hotel_opinions = hotel_opinions.drop(columns=['tipo_servicio'])
hotel_opinions.rename(columns={'nombre_servicio': 'hotel_nombre'}, inplace=True)
hotel_opinions.head()


#Now create the score for the rating of the last year
#1. turn fecha into datetime to filter the last year
hotel_opinions['fecha'] = pd.to_datetime(hotel_opinions['fecha'])
hotel_opinions['year'] = hotel_opinions['fecha'].dt.year

#2. filter the last year
hotel_opinions_2024 = hotel_opinions[hotel_opinions['year'] == 2024]
hotel_opinions_2024 = hotel_opinions_2024.drop(columns=['fecha', 'year', 'comentario'])
hotel_opinions_2024.head()

#3. Group by hotel_nombre and calculate the mean of the rating
hotel_opinions_2024 = hotel_opinions_2024.groupby('hotel_nombre').mean().round(2)
hotel_opinions_2024.reset_index(inplace=True)
hotel_opinions_2024.head()

,hotel_nombre,puntuacion
0,Alletra Boutique Hotel,3.89
1,Alletra Diamond Grand Hotel,4.31
2,Alletra Haven,4.09
3,Alletra Resort,3.50
4,Apollo Diamond Suites,4.15


In [12]:
# Identify numeric columns
numeric_cols = sustainability_data.select_dtypes(include=["number"]).columns

# Apply MinMaxScaler to numeric columns
scaler = MinMaxScaler()
scaled_numeric = pd.DataFrame(scaler.fit_transform(sustainability_data[numeric_cols]), columns=numeric_cols)

# Combine scaled numeric columns with the non-numeric ones
non_numeric = sustainability_data.drop(columns=numeric_cols).reset_index(drop=True)
sustainability_data_scaled = pd.concat([non_numeric, scaled_numeric], axis=1)

# Show result
sustainability_data_scaled.head()

,hotel_nombre,fecha,consumo_energia_kwh,residuos_generados_kg,porcentaje_reciclaje,uso_agua_m3
0,Alletra Diamond Grand Hotel,2019-01-01,0.692333,0.821,0.522222,0.9885
1,ProLiant Towers,2019-01-01,0.417667,0.319,0.477778,0.0725
2,Aruba Luxury Lodge,2019-01-01,0.941333,0.672,0.882222,0.5895
3,InfoSight Boutique Hotel,2019-01-01,0.233667,0.070,0.726667,0.9555
4,Primera Grand,2019-01-01,0.656667,0.547,0.991111,0.4535


In [13]:
# Now make a function to compute the sustainability score for each company

def compute_sustainability_score(row):
    # Define the weights for each category
    weights = {
        'consumo_energia_kwh': 0.25, 
        'residuos_generados_kg': 0.25,
        'porcentaje_reciclaje': 0.25,
        'uso_agua_m3': 0.25
    }
    
    # Compute the weighted sum of the scores
    score = (1 - row['consumo_energia_kwh']) * weights['consumo_energia_kwh'] + (1 - row['residuos_generados_kg']) * weights['residuos_generados_kg'] + row['porcentaje_reciclaje'] * weights['porcentaje_reciclaje'] + (1 - row['uso_agua_m3']) * weights['uso_agua_m3']
    
    return round(score*100, 2)

# Apply the function to the dataframe
sustainability_data_scaled['sustainability_score'] = sustainability_data_scaled.apply(compute_sustainability_score, axis=1)

# Show the result
sustainability_data_scaled.head()

,hotel_nombre,fecha,consumo_energia_kwh,residuos_generados_kg,porcentaje_reciclaje,uso_agua_m3,sustainability_score
0,Alletra Diamond Grand Hotel,2019-01-01,0.692333,0.821,0.522222,0.9885,25.51
1,ProLiant Towers,2019-01-01,0.417667,0.319,0.477778,0.0725,66.72
2,Aruba Luxury Lodge,2019-01-01,0.941333,0.672,0.882222,0.5895,41.98
3,InfoSight Boutique Hotel,2019-01-01,0.233667,0.070,0.726667,0.9555,61.69
4,Primera Grand,2019-01-01,0.656667,0.547,0.991111,0.4535,58.35


In [14]:
# Filter by the last year to make the recomendations of hotels that have the best sustainability score
#1. Make sure that the fecha column only contains dates
if sustainability_data_scaled['fecha'].dtype == 'object':
    sustainability_data_scaled['fecha'] = sustainability_data_scaled['fecha'].str.replace('d', '', regex=False)
    sustainability_data_scaled['fecha'] = pd.to_datetime(sustainability_data_scaled['fecha'], errors='coerce')

#2. Extract the year from the date
sustainability_data_scaled['año'] = sustainability_data_scaled['fecha'].dt.year 

#3. Filter by the last year
sustainability_data_scaled = sustainability_data_scaled[sustainability_data_scaled['año'] == 2024]

#4. Group by hotel_nombre and calculate the mean of the rating
sustainability_data_scaled_reduced = sustainability_data_scaled[['hotel_nombre', 'sustainability_score']].groupby('hotel_nombre').mean().round(2)

#5. Sort by the sustainability score
sustainability_data_scaled_reduced = sustainability_data_scaled_reduced.sort_values(by='sustainability_score', ascending=False).reset_index(drop=False)

#6. Show the result
sustainability_data_scaled_reduced.head()

,hotel_nombre,sustainability_score
0,Alletra Diamond Grand Hotel,51.31
1,dHCI Executive Boutique Hotel,51.09
2,Cray Villas,51.00
3,Aruba Lodge,50.96
4,Apollo Diamond Suites,50.87


In [15]:
# Merge the dataframes to get the hotels that have the best sustainability score and the rating of the last year
score_and_punctuation = pd.merge(sustainability_data_scaled_reduced, hotel_opinions_2024, left_on='hotel_nombre', right_on='hotel_nombre', how='inner')
score_and_punctuation.to_csv('data/hotels_2024_scores.csv', index=False)
score_and_punctuation

,hotel_nombre,sustainability_score,puntuacion
0,Alletra Diamond Grand Hotel,51.31,4.31
1,dHCI Executive Boutique Hotel,51.09,4.08
2,Cray Villas,51.00,4.26
3,Aruba Lodge,50.96,3.66
4,Apollo Diamond Suites,50.87,4.15
5,Apollo Towers,50.66,3.71
6,Apollo Executive Beach Resort,50.44,3.58
7,Ezmeral Grand Hotel,50.41,4.00
8,Alletra Haven,50.19,4.09
9,Pointnext Signature Residences & Suites,50.16,3.79


In [16]:
# Now we are going to make the final score which takes into account the sustainability score and the rating of the last year

# Scale puntuacion from 1–5 to 0–100
score_and_punctuation['puntuacion_scaled'] = (score_and_punctuation['puntuacion'] - 1) / (5 - 1) * 100

# Compute final score as average of both metrics (now both are 0–100)
score_and_punctuation['final_score'] = (
    score_and_punctuation['sustainability_score'] + score_and_punctuation['puntuacion_scaled']
) / 2

# Sort descending
score_and_punctuation = score_and_punctuation.sort_values(by='final_score', ascending=False).reset_index(drop=True)

# Recommendations (top 5)
top_5 = score_and_punctuation.head()
top_5

,hotel_nombre,sustainability_score,puntuacion,puntuacion_scaled,final_score
0,ProLiant Place,49.76,4.48,87.00,68.380
1,Nimble Inn,50.00,4.38,84.50,67.250
2,Alletra Diamond Grand Hotel,51.31,4.31,82.75,67.030
3,Cray Villas,51.00,4.26,81.50,66.250
4,Apollo Resort & Spa,50.12,4.21,80.25,65.185
